<a href="https://colab.research.google.com/github/TieuDiem/Resize-Image-Interpolation/blob/main/Resize_Image_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np

In [2]:
path_image = '/content/134206.jpg'
img =cv2.imread(path_image)
img.shape

(375, 500, 3)

# Interpolation With Gray Image

# Interpolation With Color Image